In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
provincias = pd.read_csv('../datasets/provicias_datos_2022.csv',
                   delimiter=';',
                   header=3)

In [3]:
provincias.head()

,PROVINCIa,población (miles),hogares (miles),ingresos laborales (miles pesos),tasa de empleo (%),superficie (km2),exportaciones (mill usd)
0,Ciudad Autónoma de Buenos Aires,3.004,1.263,211.877.384,"4,7","205,9","311,163"
1,Buenos Aires,17.292,5.738,623.250.917,"8,7","305.907,40","33024,657"
2,Catamarca,336.000,102.000,8.460.986,"7,3","101.486,10","201,03"
3,Chaco,3.477,1.251,122.790.341,"6,5","99.763,30","516,336"
4,Chubut,967.000,287.000,24.733.558,"5,3","224.302,30","2575,372"


# Limpieza de datos

Notamos que hay un error en el nombre del columna de tasa de empleo, en realidad se trata de la tasa de **desempleo**. Asumimos esto porque no tiene sentido que, por ejemplo, Buenos Aires tenga un $8.7\%$ de empleo registrado.

In [4]:
provincias.rename(
    columns={
        'PROVINCIa': 'provincia',
        'población (miles)': 'poblacion_miles',
        'hogares (miles)': 'hogares_miles',
        'ingresos laborales (miles pesos)': 'ingresos_miles',
        'tasa de empleo (%)': 'tasa_desempleo',
        'superficie (km2)': 'superficie_km2',
        'exportaciones (mill usd)': 'exportaciones_usd_millones'
    },
    inplace=True
)

Multiplicamos por mil ciertas columnas porque en la carga se leyeron como números con decimales, por ejemplo en la primer entrada el campo `poblacion` es `3.004` cuando debería ser `3004`.

Además una de las entradas tiene este campo vacío, por ahora lo asumimos como `0` para poder castear la columna y luego lo imputaremos.

In [5]:
provincias.poblacion_miles = \
    provincias.poblacion_miles\
        .apply(lambda x: x * 1000 if x < 100 else x)\
        .fillna(0).round().astype('int64')

In [6]:
provincias.hogares_miles = \
    provincias.hogares_miles\
        .apply(lambda x: x * 1000 if x < 10 else x)\
        .round().astype('int64')

Una de las entradas no tiene valor en el campo `ingresos_miles`, primero la reemplazamos por `0` para poder castear la columna a tipo entero, después imputaremos ese valor faltante.

In [7]:
provincias.ingresos_miles = \
    provincias.ingresos_miles\
    .str.replace('.', '', regex=False)\
    .fillna(0)\
    .astype('int64')

In [8]:
provincias.tasa_desempleo = \
    provincias.tasa_desempleo\
        .str.replace(',', '.', regex=False)\
        .astype('float64')

In [9]:
provincias.superficie_km2 = \
    provincias.superficie_km2\
        .str.replace('.', '', regex=False)\
        .str.replace(',', '.', regex=False)\
        .astype('float64')

En el caso de las exportaciones decidimos redondearlo a dos decimales.

In [10]:
provincias.exportaciones_usd_millones = \
    provincias.exportaciones_usd_millones\
    .str.replace(',', '.', regex=False)\
    .astype(float).round(2)

Para imputar la población de San Luis decidimos hacer un promedio de las provincias de la región de Cuyo, es decir usaremos el promedio entre las poblaciones de La Rioja, San Juan y Mendoza.

In [11]:
provincias_cuyo = ['La Rioja', 'San Juan', 'Mendoza']
media_cuyo = provincias\
                [provincias.provincia.isin(provincias_cuyo)]\
                .poblacion_miles.mean().astype('int64')

In [12]:
provincias.loc[provincias.provincia == 'San Luis', 'poblacion_miles'] = \
    media_cuyo

Para imputar el ingreso faltante de Río Negro vamos a hacer el promedio de los ingresos de las otras provincias de la región.

In [13]:
provincias_patagonia = ['Neuquén', 'Chubut',
                        'Santa Cruz', 'Tierra del Fuego']

media_patagonia = provincias\
                    [provincias.provincia.isin(provincias_patagonia)]\
                    .ingresos_miles.mean().astype('int64')

In [14]:
provincias.loc[provincias.provincia == 'Río Negro', 'ingresos_miles'] = \
    media_patagonia

# Exploración de datos

In [15]:
provincias

,provincia,poblacion_miles,hogares_miles,ingresos_miles,tasa_desempleo,superficie_km2,exportaciones_usd_millones
0,Ciudad Autónoma de Buenos Aires,3004,1263,211877384,4.7,205.9,311.16
1,Buenos Aires,17292,5738,623250917,8.7,305907.4,33024.66
2,Catamarca,336,102,8460986,7.3,101486.1,201.03
3,Chaco,3477,1251,122790341,6.5,99763.3,516.34
4,Chubut,967,287,24733558,5.3,224302.3,2575.37
5,Córdoba,1044,321,22744468,2.6,164707.8,12851.98
6,Corrientes,591,205,30606036,3.6,89123.3,273.18
7,Entre Ríos,1252,431,40510249,6.4,78383.7,1911.90
8,Formosa,509,148,9586123,3.3,75488.3,40.61
9,Jujuy,696,197,17044750,5.1,53244.2,873.44


# Conclusiones